In [3]:
#!pip install transformers

In [5]:
#!pip install flashtext

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9298 sha256=e0fb0650639298e5ee0d2d6b81f59903c30ac562e8c1ec63ed6d3ca86b966b2d
  Stored in directory: c:\users\administrator\appdata\local\pip\cache\wheels\65\3c\c7\44672c5062c16d05760b1eaddbf611d2f6a4b715c6d6777418
Successfully built flashtext


In [12]:
#!pip install torch --upgrade

In [21]:
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
     ---------------------------------------- 4.9/4.9 MB 649.2 kB/s eta 0:00:00
     ---------------------------------------- 2.5/2.5 MB 1.1 MB/s eta 0:00:00


In [1]:
import pandas as pd
from transformers import pipeline
import nltk
from nltk.corpus import stopwords
from transformers import pipeline, set_seed
import random
from flashtext import KeywordProcessor
from ast import literal_eval
import re

In [6]:
sentence1 = "Must be an extrovert with an innate quality of easily connecting with people."
sentence2 = "You're self-motivated and decisive, but willing to make changes with minimal grumbling when the client demands it."
sentence3 = "Ideal candidates will be caring, compassinate, adaptable, flexible, commited to high standards of care and possess excelent communication skills both verbal and written."
sentence4 = "To be considered for this position you must be energetic, friendly, compassionat and detail oriented."
sentence5 = "We are looking for a young and driven candidate who can bring innovation into the organization."
sentence6 = "Are you the master of technology and passionate person we are looking for?"

# Use BERT for masking arbitrary words

In [2]:
stop_words = set(stopwords.words('english'))

In [3]:
model = pipeline('fill-mask', model='bert-base-uncased')

d:\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Administrator\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relat

In [4]:
def mask_random_words(sentence, stop_words, biased_phrase, masked_result=0):
    augmented_sentence = sentence
    temp = ""
    for _ in range(5):
        not_masked =  True
        for word in augmented_sentence.split(" "):
            random_num = random.random()
            if word not in stop_words and word not in biased_phrase and random_num <= 0.2 and not_masked:
                temp += "[MASK] "
                not_masked = False
            else:
                temp += word + " "
        if "[MASK]" in temp:
            result = model(temp)
            temp = temp.replace("[MASK]", result[masked_result]['token_str'])
        augmented_sentence = temp[:-1]
        temp = ""
            
    return augmented_sentence

In [7]:
print(sentence2)
mask_random_words(sentence2, stop_words, "decisive")

You're self-motivated and decisive, but willing to make changes with minimal grumbling when the client demands it.


'very self-motivated and decisive, but able to make decisions with minimal grumbling when the client demands it.'

# Use GPT-2 to finish the sentence for you

In [10]:
generator = pipeline('text-generation', model='gpt2')
set_seed(42)

In [11]:
def finish_sentence(sentence, num_generations=5, max_length=25):
    generated_sentences = []
    temp_generations = generator(sentence, max_length=max_length, num_return_sequences=num_generations)
    for generations in temp_generations:
        temp = generations['generated_text']
        temp = temp.split('.')[0]
        temp = temp + '.'
        generated_sentences.append(temp)
    return generated_sentences

In [12]:
finish_sentence("We are looking for a young and driven candidate")

d:\anaconda3\lib\site-packages\transformers\generation\utils.py:1201: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['We are looking for a young and driven candidate who has a sense of being a member of a community, having an engaging relationship.',
 'We are looking for a young and driven candidate to carry out the best and most complete service in the areas most likely to be.',
 'We are looking for a young and driven candidate, willing to address both the needs of current and future generations of workers at the.',
 'We are looking for a young and driven candidate, one who has proven himself well and well in several of the last two elections.',
 'We are looking for a young and driven candidate who will take our leadership in all sectors of the state and work toward economic prosperity.']

# Use flashtext to replace words in sentence